In [96]:
!pip install tensorflow

  Using cached https://files.pythonhosted.org/packages/ed/11/037887c5cbac5af3124050fb6348e67caa038734cc9673b11c31c8939072/tensorflow-1.14.0-cp37-cp37m-macosx_10_11_x86_64.whl


In [97]:
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.datasets import mnist
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)


def train_model():
    input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same', name='encoder')(x)

    # at this point the representation is (4, 4, 8) i.e. 128-dimensional

    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

    autoencoder.fit(x_train_noisy, x_train,
                    epochs=20,
                    batch_size=128,
                    shuffle=True,
                    validation_data=(x_test_noisy, x_test),
                    callbacks=[TensorBoard(log_dir='/tmp/tb', histogram_freq=0, write_graph=False)])

    autoencoder.save('autoencoder.h5')

train_model()

W0828 08:53:45.940901 4773139904 deprecation_wrapper.py:119] From /Users/flatironschool/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 60000 samples, validate on 10000 samples


W0828 08:53:47.434078 4773139904 deprecation_wrapper.py:119] From /Users/flatironschool/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0828 08:53:47.438497 4773139904 deprecation_wrapper.py:119] From /Users/flatironschool/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:856: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/20
60000/60000 [==============================] - 107s 2ms/step - loss: 0.2448 - val_loss: 0.1955
Epoch 2/20
60000/60000 [==============================] - 87s 1ms/step - loss: 0.1830 - val_loss: 0.1749
Epoch 3/20
60000/60000 [==============================] - 104s 2ms/step - loss: 0.1652 - val_loss: 0.1568
Epoch 4/20
60000/60000 [==============================] - 87s 1ms/step - loss: 0.1557 - val_loss: 0.1474
Epoch 5/20
60000/60000 [==============================] - 120s 2ms/step - loss: 0.1494 - val_loss: 0.1446
Epoch 6/20
60000/60000 [==============================] - 106s 2ms/step - loss: 0.1455 - val_loss: 0.1392
Epoch 7/20
60000/60000 [==============================] - 92s 2ms/step - loss: 0.1425 - val_loss: 0.1431
Epoch 8/20
60000/60000 [==============================] - 87s 1ms/step - loss: 0.1405 - val_loss: 0.1395
Epoch 9/20
60000/60000 [==============================] - 95s 2ms/step - loss: 0.1390 - val_loss: 0.1384
Epoch 10/20
60000/60000 [==========================

In [93]:
import numpy as np
from keras.models import Model
from keras.datasets import mnist
import cv2
from keras.models import load_model
from sklearn.metrics import label_ranking_average_precision_score
import time

print('Loading mnist dataset')
t0 = time.time()
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)
t1 = time.time()
print('mnist dataset loaded in: ', t1-t0)

print('Loading model :')
t0 = time.time()
# Load previously trained autoencoder
autoencoder = load_model('autoencoder.h5')
t1 = time.time()
print('Model loaded in: ', t1-t0)

IsADirectoryError: [Errno 21] Is a directory: 'referenceImages/'

In [92]:
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
print(type(img))
# convert to numpy array
img_array = img_to_array(img)
print(img_array.dtype)
print(img_array.shape)
# convert back to image
img_pil = array_to_img(img_array)
print(type(img))

<class 'PIL.JpegImagePlugin.JpegImageFile'>
float32
(1032, 403, 3)
<class 'PIL.JpegImagePlugin.JpegImageFile'>


In [ ]:
# example of saving an image with the Keras API
from keras.preprocessing.image import save_img
# load image as as grayscale
img = load_img('bondi_beach.jpg', grayscale=True)
# convert image to a numpy array
img_array = img_to_array(img)
# save the image with a new filename
save_img('bondi_beach_grayscale.jpg', img_array)
# load the image to confirm it was saved correctly
img = load_img('bondi_beach_grayscale.jpg')
print(type(img))
print(img.format)
print(img.mode)
print(img.size)
img.show()

In [86]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [90]:
len(y_train)

60000

In [81]:
(X_train) = 'referenceImages/'
(X_test) =  'queryImages/'

In [83]:
X_train

'referenceImages/'

In [82]:
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

AttributeError: 'str' object has no attribute 'astype'

In [84]:
input_img = Input(shape=(28,28,1))
x = Conv2D(16,(3,3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(8,(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(8,(3,3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same', name='encoder')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

W0828 04:48:53.021654 4773139904 deprecation_wrapper.py:119] From /Users/flatironschool/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0828 04:48:53.186815 4773139904 deprecation_wrapper.py:119] From /Users/flatironschool/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0828 04:48:53.305834 4773139904 deprecation_wrapper.py:119] From /Users/flatironschool/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0828 04:48:53.383330 4773139904 deprecation_wrapper.py:119] From /Users/flatironschool/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d i

In [43]:
autoencoder.fit(X_train, X_train, epochs=2, batch_size=32, callbacks=None )

In [44]:
label = imagePath.split(os.path.sep)[-1].split("-")[0]

In [45]:
label

'1792'

In [28]:
import ntpath
ntpath.basename("a/b/c")

'c'

In [29]:
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

In [30]:
path_leaf(label)

'1792-bottle-in-bond-[size].png'

In [46]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('referenceImages/') if isfile(join('referenceImages/', f))]

In [65]:
labels = [i.split('-', 1)[0] for i in onlyfiles]

In [69]:
labels = [i.split(' ', 1)[0] for i in labels]

In [71]:
labels = [i.split('_', 1)[0] for i in labels]

In [73]:
len(labels)

755

In [75]:
len(set(labels))

366